1. 티커 리스트를 불러온다.
2. 티커 리스트에 맞는 

In [39]:
import pandas as pd
import requests as rq
from io import BytesIO
import zipfile

In [40]:
api_key = "d435e3001b6f5c73238e222815178a4d77953e7b"

In [41]:
codezip_url = f'''https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={api_key}'''
codezip_data = rq.get(codezip_url)
codezip_data.headers

{'Cache-Control': 'no-cache, no-store', 'Connection': 'keep-alive', 'Set-Cookie': 'WMONID=QkmrLR-FKrb; Expires=Wed, 29-May-2024 9:30:47 GMT; Path=/', 'Pragma': 'no-cache', 'Expires': '0', 'Content-Transfer-Encoding': 'binary', 'Content-Disposition': ': attachment; filename=CORPCODE.zip', 'Date': 'Tue, 30 May 2023 00:30:47 GMT', 'Content-Type': 'application/x-msdownload;charset=UTF-8', 'Content-Length': '1710152'}

In [42]:
codezip_data.headers['Content-Disposition']

': attachment; filename=CORPCODE.zip'

In [43]:
codezip_file = zipfile.ZipFile(BytesIO(codezip_data.content))
codezip_file.namelist()

['CORPCODE.xml']

In [44]:
code_data = codezip_file.read('CORPCODE.xml').decode('utf-8')

In [45]:
import xmltodict
import json

data_odict = xmltodict.parse(code_data)
data_dict = json.loads(json.dumps(data_odict))
data = data_dict.get('result').get('list')
corp_list = pd.DataFrame(data)

In [46]:
corp_list.head(10)

,corp_code,corp_name,stock_code,modify_date
0,00434003,다코,None,20170630
1,00434456,일산약품,None,20170630
2,00430964,굿앤엘에스,None,20170630
3,00432403,한라판지,None,20170630
4,00388953,크레디피아제이십오차유동화전문회사,None,20170630
5,00179984,연방건설산업,None,20170630
6,00420143,브룩스피알아이오토메이션잉크,None,20170630
7,00401111,매경아이비아이,None,20170630
8,00435534,캐드뱅크,None,20170630
9,00430186,엠와이오피삼차유동화전문유한회사,None,20170630


In [47]:
corp_list = corp_list[~corp_list.stock_code.isin([None])].reset_index(drop=True)
corp_list.head(10)

,corp_code,corp_name,stock_code,modify_date
0,00260985,한빛네트,036720,20170630
1,00264529,엔플렉스,040130,20170630
2,00358545,동서정보기술,055000,20170630
3,00231567,애드모바일,032600,20170630
4,00247939,씨모스,037600,20170630
5,00359614,리더컴,056140,20170630
6,00153551,허메스홀딩스,012400,20170630
7,00344746,유티엑스,045880,20170630
8,00261188,글로포스트,037830,20170630
9,00268020,쏠라엔텍,030390,20170630


In [48]:
corp_list.to_csv("./corp_codes.csv")

In [49]:
corp_code = '00126380'
bsns_year = '2015'
reprt_code = '11011'

url_fs = f'''https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?crtfc_key={api_key}
&corp_code={corp_code}&bsns_year={bsns_year}&reprt_code={reprt_code}&fs_div=OFS'''

fs_data_ss = rq.get(url= url_fs)
fs_data_ss_df = fs_data_ss.json().get('list')
fs_data_ss_df = pd.DataFrame(fs_data_ss_df)
fs_data_ss_df[fs_data_ss_df['account_nm']=='이자의 지급']['thstrm_amount'].values[0]

'153195000000'

In [50]:
kospi = pd.read_csv('kospi.csv')
columns_to_keep = ['회사명', '종목코드', '회계년도', '거래월', '종가']
kospi = kospi[columns_to_keep]
kospi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18345 entries, 0 to 18344
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   회사명     18345 non-null  object 
 1   종목코드    18345 non-null  int64  
 2   회계년도    18345 non-null  object 
 3   거래월     18345 non-null  float64
 4   종가      18345 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 716.7+ KB
